# (7.7) Estimadores de estado

No assunto anterior vimos que a ação de controle no espaço de estados é proporcional aos estados do sistema. Para isso ser possível de calcular, precisamos ter esses sinais disponíveis.

Na prática, uma medição direta dos estados é raramente viável. O número de sensores pode ser grande, o que tornaria o projeto muito caro. Ou simplesmentes, os estados não são possíveis de medir.

Para contornar a situação usamos um sistema auxiliar, um subsistema do controlador, que é responsável de fornecer uma estimativa dos estados reais. Esse sistema é chamado de **estimador** ou **observador de estados**.

Vamos supor que os estados reais sejam $\mathbf{x}$ e a respectiva estimativa seja $\hat{\mathbf{x}}$. Idealmente, queremos que $\hat{\mathbf{x}}\approx \mathbf{x}$.

Suponha que o sistema real seja:
$$
\begin{align*}
    \dot{\mathbf{x}} &= \mathbf{Fx+G}u\\
    y &= \mathbf{Hx} + Du
\end{align*}
$$

O estimador é um sistema que tenta "imitar" o original, usando uma dinâmica parecida:
$$
\begin{align*}
    \dot{\mathbf{\hat{x}}} &= \mathbf{F\hat{x}+G}u+\mathbf{L}(y-\mathbf{H\hat{x}})
\end{align*}
$$

A interpretação dessa equação é a seguinte
* O termo $\mathbf{F\hat{x}+G}u$ é uma tentativa de "imitar" a equação original do sistema.
* O termo $\mathbf{L}(y-\mathbf{H\hat{x}})$ é um fator de correção do anterior.
    * O termo $\mathbf{H\hat{x}}$ representa uma estimativa da saída do sistema (é a matriz de saída $\mathbf{H}$ vezes os estados estimados)
    * O termo $y-\mathbf{H\hat{x}}$ representa, portanto, o erro entre a saída real e a saída estimada.
    * O vetor coluna $\mathbf{L}$ funciona como um ganho proporcional.

Assim, vemos que o estimador é um sistema que "imita" a dinâmica do sistema original, porém, ele corrige o erro da dinâmica usando uma parcela proporcional ao erro de estimativa da saída (que é um sinal que nós realmente conseguimos medir).

Esse termo de correção é o que faz o estimador funcionar. Projetar o estimador é essencialmente calcular o vetor $\mathbf{L}$, chamado de *ganho do estimador*. 

Se o ganho do estimador for bem projetado, a diferença entre $\mathbf{x}-\mathbf{\hat{x}}$ cairá rapidamente com o tempo. Ou seja, passado um longo tempo (isto é, em regime permanente), $\mathbf{x}$ e $\mathbf{\hat{x}}$ terão os mesmos valores. Matematicamente representamos $\mathbf{x}\rightarrow \mathbf{\hat{x}}$.

<p align="center">
<img src="Fig7.28.svg" width="80%">
</p>
<p align="center">
**Esquemático em blocos do funcionamento do estimador.**
</p>


A síntese do estimador é feita da seguinte maneira. Seja $e = \mathbf{x}-\mathbf{\hat{x}}$ o erro de estimação entre os estados. Combinando a dinâmica do sistema e do estimador, podemos mostrar que:
$$
\begin{align*}
    \dot{e} &= \mathbf{(F-LH)e}
\end{align*}
$$

Esse é um sistema autônomo (sem entrada), que só depende das condições iniciais. Ele só possui a matriz de estados, que se for estável, fará com que o estado inicial decaia a zero, qualquer que ele seja.

Em outras palavras, mesmo que não conheçamos o estado inicial do sistema para alimentar o estimador, o erro ainda assim irá para zero em regime permanente. 

Para que isso aconteça, basta que a matriz $\mathbf{(F-LH)}$ seja estável, isto é, tenha todos os autovalores no SPE. 

Esse é um problema semelhante ao da regulação de estados. A única diferença é a posição das matrizes. No problema de regulação temos $\mathbf{F-GK}$.
<div align="center">

|            |    Regulador    |    Estimador    |
|------------|:---------------:|:---------------:|
| Determinar |   $\mathbf{K}$  |   $\mathbf{L}$  |
| Dimensões  |   $1\times n$   |   $n\times 1$   |
| Equação    | $\mathbf{F-GK}$ | $\mathbf{F-LH}$ |
</div>

Devido às semelhanças, podemos usar as mesmas estratégias e funções Python para projetar o estimador. 

## Exemplo:

<p align="center">
<img src="Ex7.25.svg" width="80%">
</p>

In [2]:
import numpy as np
import sympy as sp

s = sp.symbols('s')
w0 = sp.symbols('\omega_0')
l1,l2 = sp.symbols(['l_1','l_2'])


100*\omega_0**2 + 20*\omega_0*s + s**2

In [4]:

# Matrizes do sistema
F = sp.Matrix([[0,1],[-w0**2,0]])
H = sp.Matrix([1,0]).T
L = sp.Matrix([[l1],[l2]])

# Matriz de malha fechada do observador
M = F-sp.MatMul(L,H)

# Polinômio desejado
p_desejado = sp.expand((s+10*w0)**2)
# Polinômio do observador (F-LH)
p_obsv = sp.det(s*sp.eye(2)-M)

p_obsv

\omega_0**2 + l_1*s + l_2 + s**2

In [32]:
sp.collect(p_desejado,s)

100*\omega_0**2 + 20*\omega_0*s + s**2

In [33]:
sp.collect(p_malha,s)

\omega_0**2 + l_1*s + l_2 + s**2

In [34]:
eq1 = sp.Eq(p_malha.coeff(s,1),p_desejado.coeff(s,1))
eq1

Eq(l_1, 20*\omega_0)

In [35]:
eq2 = sp.Eq(p_malha.coeff(s,0),p_desejado.coeff(s,0))
eq2

Eq(\omega_0**2 + l_2, 100*\omega_0**2)

In [36]:
sol = sp.solve([eq1,eq2],L)

In [37]:
sol[l1]

20*\omega_0

In [38]:
sol[l2]

99*\omega_0**2

**Solução numérica**

Se adotarmos $\omega_0=1$, podemos resolver o mesmo problema usando a função de posicionamento de polos *acker()*. De fato, isso é possível, porque o problema do observador é muito semelhante ao do regulador, mudando apenas os seguintes parâmetros.
* Usamos $\mathbf{F}^T$ ao invés de $\mathbf{F}$
* Usamos $\mathbf{H}^T$ ao invés de $\mathbf{G}$
* Transpomos o resultado para obter o ganho do observador na forma de vetor coluna

In [5]:
import control as ct

w0=1
F = np.array([[0,1],[-w0**2,0]])
H = np.array([[1,0]])

L = ct.acker(A=F.T,B=H.T,poles=[-10*w0,-10*w0]).T
print(L)

[[20.]
 [99.]]
